In [1]:
source ../config_duke.sh -v
source ../config_load_module_bedtools.sh

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc



## set range

In [2]:
FD_FRG=${FD_RES}/KS91_K562_ASTARRseq/fragment
FD_COV=${FD_RES}/KS91_K562_ASTARRseq/coverage
ls ${FD_RES}/KS91_K562_ASTARRseq

coverage  fragment


In [3]:
ls ${FD_FRG}

KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep5.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep6.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Output_rep1.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Output_rep2.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Output_rep3.GATA1.unstranded.bed
KS91_K562_hg38_ASTARRseq_Output_rep4.GATA1.unstranded.bed


In [4]:
### KS91_K562_hg38_ASTARRseq_Output_rep1.GATA1.unstranded.bed
### KS91_K562_hg38_ASTARRseq_Output_rep1.GATA1.unstranded.perbase.tsv.gz

FP_BEDS=($(ls ${FD_FRG}/KS91*))
for FP_INP in ${FP_BEDS[@]}; do
    FN_INP=$(basename ${FP_INP})
    PREFIX=${FN_INP%.*}
    FN_OUT=${PREFIX}.perbase.tsv.gz
    echo ${FP_INP}
    echo ${FN_OUT}
done

/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.perbase.tsv.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep2.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep2.GATA1.unstranded.dedups.perbase.tsv.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep3.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep3.GATA1.unstranded.dedups.perbase.tsv.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.dedups.perbase.tsv.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep5.GATA1.unstranded.d

In [5]:
FN_BEDS=($(ls ${FD_FRG}))

IDX=0
FN_BED=${FN_BEDS[${IDX}]}
echo ${FN_BED}

IDX=3
FN_BED=${FN_BEDS[${IDX}]}
echo ${FN_BED}

KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.bed
KS91_K562_hg38_ASTARRseq_Input_rep4.GATA1.unstranded.dedups.bed


## RUN

In [83]:
TAB="$(printf '\t')"
cat > ${FD_COV}/region_GATA1.bed << EOF
chrX${TAB}47786400${TAB}49786750
EOF

In [84]:
sbatch -p ${NODE} \
    --mem 8G \
    --array 0-9 \
    -o ${FD_LOG}/ASTARR_coverage_gata1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh

### set input and output
FD_FRG=${FD_RES}/KS91_K562_ASTARRseq/fragment
FD_COV=${FD_RES}/KS91_K562_ASTARRseq/coverage

FP_BEDS=($(ls ${FD_FRG}/KS91*))
FP_INP=${FP_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

PREFIX=${FN_INP%.*}
FN_OUT=${PREFIX}.perbase.tsv.gz
FP_OUT=${FD_COV}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo "Input:             " ${FP_INP}
echo "Output:            " ${FP_OUT}
echo

### show first few lines of input
echo ${FP_INP}
cat ${FP_INP} | head -3
echo

### execute
bedtools coverage \
    -a ${FD_COV}/region_GATA1.bed \
    -b ${FP_INP} \
    -d  |\
    awk -v OFS='\t' {'print $1, $2+$4-1, $5'} |\
    gzip -c \
    > ${FP_OUT}

### show first few lines of output
echo ${FP_OUT}
zcat ${FP_OUT} | head -3
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 27843882


In [85]:
cat ${FD_LOG}/ASTARR_coverage_gata1.0.txt

Slurm Array Index:  0
Time Stamp:         03-21-22+14:26:24
Input:              /gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.bed
Output:             /gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/coverage/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.perbase.tsv.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/fragment/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.bed
chrX	47787165	47787363	chrX_47787165_47787363
chrX	47787557	47787773	chrX_47787557_47787773
chrX	47787593	47787783	chrX_47787593_47787783

/gpfs/fs1/data/reddylab/Kuei/out/proj_encode_fcc/KS91_K562_ASTARRseq/coverage/KS91_K562_hg38_ASTARRseq_Input_rep1.GATA1.unstranded.dedups.perbase.tsv.gz
chrX	47786400	0
chrX	47786401	0
chrX	47786402	0


Done!
Run Time: 5 seconds


## Check results

In [33]:
cat ${FD_FRG}/KS91_K562_hg38_ASTARRseq_Output_rep1.GATA1.unstranded.bed | head -n 20

chrX	47787189	47787363	chrX_47787189_47787363
chrX	47787189	47787363	chrX_47787189_47787363
chrX	47787189	47787363	chrX_47787189_47787363
chrX	47787189	47787363	chrX_47787189_47787363
chrX	47787189	47787363	chrX_47787189_47787363
chrX	47787189	47787363	chrX_47787189_47787363
chrX	47788545	47789154	chrX_47788545_47789154
chrX	47788545	47789154	chrX_47788545_47789154
chrX	47788545	47789154	chrX_47788545_47789154
chrX	47788545	47789154	chrX_47788545_47789154
chrX	47788545	47789154	chrX_47788545_47789154
chrX	47788545	47789154	chrX_47788545_47789154
chrX	47788659	47788978	chrX_47788659_47788978
chrX	47788659	47788978	chrX_47788659_47788978
chrX	47788659	47788978	chrX_47788659_47788978
chrX	47788659	47788978	chrX_47788659_47788978
chrX	47788659	47788979	chrX_47788659_47788979
chrX	47788659	47788979	chrX_47788659_47788979
chrX	47788659	47788979	chrX_47788659_47788979
chrX	47788659	47788979	chrX_47788659_47788979
cat: write error: Broken pipe


In [25]:
zcat ${FD_COV}/KS91_K562_hg38_ASTARRseq_Output_rep1.GATA1.unstranded.perbase.tsv.gz |\
    head -n 800 |\
    tail -n 20


gzip: chrX	47787180	0
chrX	47787181	0
stdout: Broken pipe
chrX	47787182	0
chrX	47787183	0
chrX	47787184	0
chrX	47787185	0
chrX	47787186	0
chrX	47787187	0
chrX	47787188	0
chrX	47787189	6
chrX	47787190	6
chrX	47787191	6
chrX	47787192	6
chrX	47787193	6
chrX	47787194	6
chrX	47787195	6
chrX	47787196	6
chrX	47787197	6
chrX	47787198	6
chrX	47787199	6


In [34]:
zcat ${FD_COV}/KS91_K562_hg38_ASTARRseq_Output_rep1.GATA1.unstranded.perbase.tsv.gz |\
    head -n 2270 |\
    tail -n 20


gzip: chrX	47788650	6
chrX	47788651	6
stdout: Broken pipe
chrX	47788652	6
chrX	47788653	6
chrX	47788654	6
chrX	47788655	6
chrX	47788656	6
chrX	47788657	6
chrX	47788658	6
chrX	47788659	28
chrX	47788660	29
chrX	47788661	29
chrX	47788662	30
chrX	47788663	30
chrX	47788664	30
chrX	47788665	30
chrX	47788666	30
chrX	47788667	30
chrX	47788668	30
chrX	47788669	30
